In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
import math
from xgboost import XGBRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier, Fourier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.compose import make_column_selector
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create features and shift output
def make_lags(ts, lags, lead_time=1):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)

def make_multistep_target(ts, steps):
    return pd.concat(
        {f'y_step_{i + 1}': ts.shift(-i)
         for i in range(steps)},
        axis=1)

def make_lags_transformer(n_lags):
    return FunctionTransformer(lambda x: make_lags(x, n_lags))

# Encode these as sine/cosine features to capture cyclical nature
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def data_shift_transformer():
    return FunctionTransformer(lambda x: x.dropna())

In [ ]:
# Methods for feature engineering
def lagplot(x, y=None, lag=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(lag)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    ax.set(title=f"Lag {lag}", xlabel=x_.name, ylabel=y_.name)
    return ax


def plot_lags(x, y=None, lags=6, nrows=1, lagplot_kwargs={}, **kwargs):
    import math
    kwargs.setdefault('nrows', nrows)
    kwargs.setdefault('ncols', math.ceil(lags / nrows))
    kwargs.setdefault('figsize', (kwargs['ncols'] * 2, nrows * 2 + 0.5))
    fig, axs = plt.subplots(sharex=True, sharey=True, squeeze=False, **kwargs)
    for ax, k in zip(fig.get_axes(), range(kwargs['nrows'] * kwargs['ncols'])):
        if k + 1 <= lags:
            ax = lagplot(x, y, lag=k + 1, ax=ax, **lagplot_kwargs)
            ax.set_title(f"Lag {k + 1}", fontdict=dict(fontsize=14))
            ax.set(xlabel="", ylabel="")
        else:
            ax.axis('off')
    plt.setp(axs[-1, :], xlabel=x.name)
    plt.setp(axs[:, 0], ylabel=y.name if y is not None else x.name)
    fig.tight_layout(w_pad=0.1, h_pad=0.1)
    return fig

def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [ ]:
# Plotting helpers
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)
%config InlineBackend.figure_format = 'retina'


def plot_multistep(y, every=1, ax=None, palette_kwargs=None):
    palette_kwargs_ = dict(palette='husl', n_colors=16, desat=None)
    if palette_kwargs is not None:
        palette_kwargs_.update(palette_kwargs)
    palette = sns.color_palette(**palette_kwargs_)
    if ax is None:
        fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', palette))
    for date, preds in y[::every].iterrows():
        preds.index = pd.period_range(start=date, periods=len(preds))
        preds.plot(ax=ax)
    return ax

In [ ]:
# Plotting helpers
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)
%config InlineBackend.figure_format = 'retina'


def plot_multistep(y, every=1, ax=None, palette_kwargs=None):
    palette_kwargs_ = dict(palette='husl', n_colors=16, desat=None)
    if palette_kwargs is not None:
        palette_kwargs_.update(palette_kwargs)
    palette = sns.color_palette(**palette_kwargs_)
    if ax is None:
        fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', palette))
    for date, preds in y[::every].iterrows():
        preds.index = pd.period_range(start=date, periods=len(preds))
        preds.plot(ax=ax)
    return ax

In [ ]:
def apply_pca(X, standardize=True):
    # Standardize
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Create principal components
    pca = PCA()
    X_pca = pca.fit_transform(X)
    
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    
    # Create loadings
    loadings = pd.DataFrame(
        pca.components_.T,  # transpose the matrix of loadings
        columns=component_names,  # so the columns are the principal components
        index=X.columns,  # and the rows are the original features
    )
    return X_pca, loadings, pca

def apply_pca_transformer():
    return FunctionTransformer(lambda x: apply_pca(x)[0])

In [ ]:
def round_to_nearest_n(data, rounding_level):
    return rounding_level * round(data / rounding_level)


In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

# Preliminary Data Transforms

In [ ]:
# Get data
kaituna_data = pd.read_csv('../input/kaituna-20182022/kaituna_data_2018-01-01_2022-10-19.csv',parse_dates=["TimeStamp"])

kaituna_data["TimeStamp"] = pd.to_datetime(kaituna_data['TimeStamp'], utc=True)
kaituna_data = kaituna_data.set_index("TimeStamp")
kaituna_data.info()

In [ ]:
#Average gate level
kaituna_data["AverageGate"] = kaituna_data[["Gate1", "Gate2", "Gate3"]].median(axis=1)

# Average gate, rounded to the nearest 100
gate_resolution_level = 100
kaituna_data["AverageGateOrdinal"] = round_to_nearest_n(kaituna_data["AverageGate"], gate_resolution_level)


# Extract hour, day, month, year from TimeStamp
kaituna_data["Hour"] = kaituna_data.index.hour
kaituna_data["DayOfWeek"] = kaituna_data.index.day_of_week
kaituna_data["DayOfYear"] = kaituna_data.index.dayofyear
kaituna_data["Month"] = kaituna_data.index.month
kaituna_data["Year"] = kaituna_data.index.year

In [ ]:
#Define categorical columns
categorical_cols = ['Hour', 'DayOfWeek', 'Month', 'Year', 'AverageGateOrdinal']
levels = []
for col in categorical_cols:
    kaituna_data[col] = kaituna_data[col].astype('int')

#Numerical columns
numerical_cols = ['FlowRate','AverageGate','Rainfall','LakeLevel']

# Data Exploration
Initial data exploration - sanity checks and determining suitable features/outputs

In [ ]:
# Exploration of variables
sns.lineplot(x=kaituna_data.index,y=kaituna_data["AverageGate"])
plt.figure()
sns.lineplot(x=kaituna_data.index,y=kaituna_data["AverageGateOrdinal"])
plt.figure()
sns.lineplot(x=kaituna_data.index,y=kaituna_data["FlowRate"])
plt.figure()
sns.lineplot(x=kaituna_data.index,y=kaituna_data["LakeLevel"])
plt.figure()
sns.lineplot(x=kaituna_data.index,y=kaituna_data["Rainfall"])


# Visualise gate levels
plt.figure()
palette = ['tab:blue', 'tab:green', 'tab:red']
gates_to_cumecs = sns.scatterplot(
    x=kaituna_data["AverageGate"], 
    y=kaituna_data["FlowRate"], 
    #hue=kaituna_data["Year"],
    #palette='Set1'
    )
plt.ylabel('Flow rate (cumecs)')
plt.xlabel("Gate Position")
plt.title("Kaituna Flow")
plt.show()

# Investigate correlation of lake level and rainfall
plt.figure()
palette = ['tab:blue', 'tab:green', 'tab:red']
gates_to_cumecs = sns.scatterplot(
    x=kaituna_data["Rainfall"], 
    y=kaituna_data["LakeLevel"], 
    #hue=kaituna_data["Year"],
    #palette='Set1'
    )
plt.ylabel('LakeLevel')
plt.xlabel("Rainfall")
plt.title("Kaituna Flow")
plt.show()

In [ ]:
# Shape
display(kaituna_data.shape)

# Describe
display(kaituna_data.describe())

# Describe ordinal variables
#print(kaituna_data[categorical_cols].describe())
display(kaituna_data.info())

display(kaituna_data.head())

# Observations
## Flow data and gate levels
This is highly seasonal. It appears that the primary trend is annual, peaking in winter and dipping in summer. However, it's not fully dependent on this, evidenced particularly by the latter months of 2022, where we had opens for several weeks.

## Rainfall
The rainfall data initially looked suspicious - 75% of data was 0mm. I checked and double checked the data loading process and it looks correct. It is possible that, because it is hourly data, that there are many more hours where it doesn't rain, compared to when it does rain.

## Case for predicting daily gate levels versus hourly
Given that so much rainfall data is actually 0, it may be preferable to try and predict the daily gate levels. This is actually favourable for a few reasons:

1. Paddlers typically understand this more than flow rates.
2. The gate levels, in my experience, rarely change in the course of a day/are set from the morning.

In saying this, if there is a gate change, say, in the afternoon, then it is possible that this will be missed by the algorithm. So let's begin with hourly and see how that goes

## Feature extraction

In [ ]:
# Get features and target
# Date features
date_features = [
    #'Hour',
    'DayOfWeek',
    'Month'
]

#Numerical features
numerical_features = [
    'Rainfall',
    'LakeLevel',
    'FlowRate'
]

#Categorical features
categorical_features = [
    'AverageGateOrdinal'
]

#Combine to create the training data for the residuals model
X_raw = kaituna_data[date_features + numerical_features + categorical_features]

display(X_raw.shape)
display(X_raw.head())
display(X_raw.info())

### Summarising into daily 

Having initially tried hourly predictions, I quickly found two things:

1. Rainfall data was hard to use as it was mostly 0s
2. The number of timesteps predicting into the future were difficult to deal with and took a long time to train. 

Daily is perhaps more useful anyway, given how kayakers plan

In [ ]:
def is_weekend(row):    
    return (row['DayOfWeek'] == 5) or (row["DayOfWeek"] == 6)

def aggregate_to_daily(hourly_data):
    X_daily = hourly_data.groupby(by=hourly_data.index.date).agg(
        Rainfall = ('Rainfall','sum'), 
        LakeLevel=('LakeLevel','mean'),
        DayOfWeek = ('DayOfWeek', lambda x:x[0]),
        Month = ('Month', lambda x:x[0]),
        AverageGateOrdinal = ('AverageGateOrdinal', lambda x:pd.Series.mode(x)[0]),
        FlowRate = ('FlowRate','median'),
        #IsWeekend = ('DayOfWeek', lambda x: 1 if (pd.Series.mode(x) == 5.0) | (pd.Series.mode(x) == 6.0) else 0)
    )

    X_daily["IsWeekend"] = X_daily.apply(is_weekend, axis=1)
    
    return X_daily

X_daily = aggregate_to_daily(X_raw)

#Set target variable
target_column="FlowRate"

X_daily.index = pd.to_datetime(X_daily.index)
X_daily.set_index(X_daily.index.to_period('D'), inplace=True)

display(X_daily.info())
display(X_daily.head())

X_daily.to_csv('kaituna_daily.csv')

In [ ]:
# Exploration of variables
plt.figure()
X_daily["FlowRate"].plot()
plt.ylabel("Flow rate (cumecs)")

plt.figure()
X_daily["LakeLevel"].plot()
plt.ylabel("Lake Level (m)")

plt.figure()
sns.histplot(data=X_daily["Rainfall"])

plt.figure()
X_daily["Rainfall"].plot()
plt.ylabel("Rainfall (mm)")

# Investigate correlation of lake level and rainfall
plt.figure()
palette = ['tab:blue', 'tab:green', 'tab:red']
gates_to_cumecs = sns.scatterplot(
    x=X_daily["Rainfall"], 
    y=X_daily["LakeLevel"], 
    #hue=X_daily["IsWeekend"],
    #palette='Set1'
    )
plt.ylabel('LakeLevel')
plt.xlabel("Rainfall")
plt.title("LakeLevel x Rainfall")
plt.show()

### Seasonal features

In [ ]:
# Check seasonality
plot_periodogram(X_daily[target_column])

# Trend feature (unused currently)
calendar_fourier = CalendarFourier(freq="A", order=1)#10 sin/cos pairs for "A"nnual seasonality
fourier = Fourier(period=365.25*4, order=1)

#Features for linear regression
dp = DeterministicProcess(
    index=X_daily.index,  # dates from the training data
    constant=False,       # dummy feature for the bias (y_intercept)
    order=0,             # the time dummy (trend)
    additional_terms = [calendar_fourier],
    drop=True,           # drop terms if necessary to avoid collinearity
)

X_seasonal_features = dp.in_sample()
display(X_seasonal_features.head())

plt.figure()
test = X_seasonal_features.index.to_timestamp()
plt.plot(test, X_seasonal_features.values)
plt.show()

In [ ]:
# Investigate if there is serial dependence
_ = plot_lags(X_daily[target_column], lags=12, nrows=2)
_ = plot_pacf(X_daily[target_column], lags=12)

#As per https://www.kaggle.com/code/ryanholbrook/time-series-as-features, let's assume >0.1 is significant correlation 

### Mutual information

#### On flow rate

In [ ]:
mi_scores = make_mi_scores(X_daily[["Rainfall","LakeLevel"]], X_daily["FlowRate"], discrete_features=False)
mi_scores

We can see that lake level contains much more information about the flowrate than the rainfall. 

#### On gate levels

In [ ]:
mi_scores = make_mi_scores(X_daily[["Rainfall","LakeLevel"]], X_daily["AverageGateOrdinal"], discrete_features=False)
mi_scores

Interestingly, lake level does not explain this as much, while rainfall explains it a bit more. This is possibly because the gate operators are predicting lake levels to rise based on rain

## PCA analysis

In [ ]:
# Exploring potential for PCA
X_pca_candidate = X_daily[["Rainfall","LakeLevel"]]
X_pca_exploration,loadings, pca = apply_pca(X_pca_candidate)

display(loadings)
plot_variance(pca);

In [ ]:
X_rainfall_lake_level_ratio = X_daily["Rainfall"] * X_daily["LakeLevel"]

sns.scatterplot(x=X_rainfall_lake_level_ratio, y=X_daily["FlowRate"])
X_rainfall_lake_level_ratio.head()

plt.figure()
X_rainfall_lakelevel_sum = X_daily["Rainfall"] + X_daily["LakeLevel"]
sns.scatterplot(x=X_rainfall_lakelevel_sum, y=X_daily["FlowRate"])
X_rainfall_lakelevel_sum.head()

We can see that there is potential for PCA features to provide information - i.e. there is some level of correlation (as expected) and PCA may be able to reduce the dimensionality of the problem.

Interestingly, using the loadings to create a new feature has revealed that there are two distinct clusters, that seem to be separated by high/low FlowRate. However, this could be the consequence of different gate levels, as there are also other less distinct clusters below

Let's examine the MI scores on each of the two candidate target variables

In [ ]:
flowrate_mi_scores = make_mi_scores(X_pca_exploration, X_daily["FlowRate"],discrete_features=False)
gate_level_mi_scores = make_mi_scores(X_pca_exploration, X_daily["AverageGateOrdinal"],discrete_features=False)

print("Flowrate Level MI:")
display(flowrate_mi_scores)
print()
print("Gate Level MI:")
display(gate_level_mi_scores)

In both cases, both PCA components have significant information about the target variable. This indicates that it may be beneficial to use PCA in the final model

### PCA with lag variables

In [ ]:
# Create lags with pipeline

# Days to look back
n_rainfall_lags = 1
n_lake_level_lags = 1
n_target_lags = 1

# Bundle preprocessing for data. We transform the date variables to cyclic, and make time series lags 
# for the lake level and rainfall data
lag_generator = ColumnTransformer(
    transformers=[
        ("target_lags", make_lags_transformer(n_target_lags), [target_column]),
        ("rainfall_lags", make_lags_transformer(n_rainfall_lags), ["Rainfall"]),
        ("lakelevel_lags", make_lags_transformer(n_lake_level_lags), ["LakeLevel"]),       
    ],
)

X_lag_features = lag_generator.fit_transform(X_daily)

In [ ]:
# Convert back to df
# Get column names
column_prefixes = [
    "Target",
    "Rainfall",
    "LakeLevel"]

column_names = []

temp_list = [column_prefixes[0] + "_lag_{}".format(i) for i in range(1,n_target_lags + 1)]
column_names.extend(temp_list)
    
# Rainfall lags
temp_list = [column_prefixes[1] + "_lag_{}".format(i) for i in range(1,n_rainfall_lags + 1)]
column_names.extend(temp_list)

# Lake level lags
temp_list = [column_prefixes[2] + "_lag_{}".format(i) for i in range(1,n_lake_level_lags + 1)]
column_names.extend(temp_list)
"""
for column_prefix in column_prefixes:
    temp_list = [column_prefix + "_lag_{}".format(i) for i in range(1,n_lags + 1)]
    column_names.extend(temp_list)
"""
#column_names.extend(["Target_lag_1"])

# Create data frame
X_lag_features_df = pd.DataFrame(X_lag_features, columns=column_names, index=X_daily.index)

display(X_lag_features_df.head())

In [ ]:
# Perform PCA on the features - we know there is already potential to use this. Also the features are
# likely to be highly correlated
X_lags_pca_candidate = X_lag_features_df.dropna()
X_lags_pca, loadings_lags, pca_lag = apply_pca(X_lags_pca_candidate)

display(loadings_lags)
plot_variance(pca_lag);

# MI scores for PCA components
pca_mi_scores = make_mi_scores(X_lags_pca, X_daily[target_column].loc[X_lags_pca_candidate.index],discrete_features=False)
#gate_level_mi_scores = make_mi_scores(X_pca, y["AverageGateOrdinal"],discrete_features=False)

display(pca_mi_scores)

In [ ]:
X_target_lag_lake_level_ratio = X_lag_features[:,0] + X_lag_features[:,2]

sns.scatterplot(x=X_target_lag_lake_level_ratio, y=X_daily["FlowRate"])
plt.xlabel("Ratio of previous lake level and previous flow rate against today's flow rate")

plt.figure()
sns.scatterplot(x=X_lag_features[:,0], y=X_daily["FlowRate"])
plt.xlabel("Previous day's flow rate")

This probably doesn't add huge amounts of new information, compared to simply plotting against the previous day's flow